In [1]:
import numpy as np
import pandas as pd
import cv2
import os

IMAGE_SIZE = (331, 331)
IMAGE_FULL_SIZE = (331, 331, 3)

train_images_folder_path = "train"
test_images_folder_path = "test"


# Load the data
train_images = np.load("train_images.npy")
train_labels = np.load("train_labels.npy")


from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
labels = le.fit_transform(train_labels)

num_of_classes = len(np.unique(labels))
print(num_of_classes)


120


In [2]:
from tensorflow.keras.utils import to_categorical

labels_for_model = to_categorical(labels, num_classes=num_of_classes)
print(labels_for_model)

c:\Users\kinga\anaconda3\envs\iowad\Lib\site-packages\h5py\__init__.py:36: UserWarning: h5py is running against HDF5 1.14.3 when it was built against 1.14.2, this may cause problems
  _warn(("h5py is running against HDF5 {0} when it was built against {1}, "


[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [3]:
from sklearn.model_selection import train_test_split
train_images = train_images / 255.0
X_train, X_test, y_train, y_test = train_test_split(train_images, labels_for_model, test_size=0.3, random_state=42)

In [4]:
del train_images
del labels
del labels_for_model

In [5]:
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.nasnet import NASNetLarge

model = NASNetLarge(input_shape=IMAGE_FULL_SIZE, weights='imagenet', include_top=False)

for layer in model.layers:
    layer.trainable = False
    print(layer.name)

flatten = Flatten()(model.output)
dense = Dense(num_of_classes, activation='softmax')(flatten)

model = Model(inputs=model.input, outputs=dense)
print(model.summary())


input_layer
stem_conv1
stem_bn1
activation
reduction_conv_1_stem_1
reduction_bn_1_stem_1
activation_1
activation_3
separable_conv_1_pad_reduction_left1_stem_1
separable_conv_1_pad_reduction_right1_stem_1
separable_conv_1_reduction_left1_stem_1
separable_conv_1_reduction_right1_stem_1
separable_conv_1_bn_reduction_left1_stem_1
separable_conv_1_bn_reduction_right1_stem_1
activation_2
activation_4
separable_conv_2_reduction_left1_stem_1
separable_conv_2_reduction_right1_stem_1
activation_5
separable_conv_2_bn_reduction_left1_stem_1
separable_conv_2_bn_reduction_right1_stem_1
separable_conv_1_pad_reduction_right2_stem_1
activation_7
reduction_add_1_stem_1
separable_conv_1_reduction_right2_stem_1
separable_conv_1_pad_reduction_right3_stem_1
activation_9
separable_conv_1_bn_reduction_right2_stem_1
separable_conv_1_reduction_right3_stem_1
separable_conv_1_reduction_left4_stem_1
activation_6
separable_conv_1_bn_reduction_right3_stem_1
separable_conv_1_bn_reduction_left4_stem_1
reduction_pad_1_

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 331, 331,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_conv1 (Conv2D) │ (None, 165, 165,  │      2,592 │ input_layer[0][0] │
│                     │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_bn1            │ (None, 165, 165,  │        384 │ stem_conv1[0][0]  │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation          │ (None, 165, 165,  │          0 │ stem_bn1[0][0]    │
│ (Activation)        │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ reduction_conv_1_s… │ (None, 165, 165,  │      4,032 │ activation[0][0]  │
│ (Conv2D)            │ 42)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ reduction_bn_1_ste… │ (None, 165, 165,  │        168 │ reduction_conv_1… │
│ (BatchNormalizatio… │ 42)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_1        │ (None, 165, 165,  │          0 │ reduction_bn_1_s… │
│ (Activation)        │ 42)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_3        │ (None, 165, 165,  │          0 │ stem_bn1[0][0]    │
│ (Activation)        │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ separable_conv_1_p… │ (None, 169, 169,  │          0 │ activation_1[0][… │
│ (ZeroPadding2D)     │ 42)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ separable_conv_1_p… │ (None, 171, 171,  │          0 │ activation_3[0][… │
│ (ZeroPadding2D)     │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ separable_conv_1_r… │ (None, 83, 83,    │      2,814 │ separable_conv_1… │
│ (SeparableConv2D)   │ 42)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ separable_conv_1_r… │ (None, 83, 83,    │      8,736 │ separable_conv_1… │
│ (SeparableConv2D)   │ 42)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ separable_conv_1_b… │ (None, 83, 83,    │        168 │ separable_conv_1… │
│ (BatchNormalizatio… │ 42)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ separable_conv_1_b… │ (None, 83, 83,    │        168 │ separable_conv_1… │
│ (BatchNormalizatio… │ 42)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_2        │ (None, 83, 83,    │          0 │ separable_conv_1… │
│ (Activation)        │ 42)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_4        │ (None, 83, 83,    │          0 │ separable_conv_1… │
│ (Activation)        │ 42)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ separable_conv_2_r… │ (None, 83, 83,    │      2,814 │ activation_2[0][

 Total params: 143,461,578 (547.26 MB)

 Trainable params: 58,544,760 (223.33 MB)

 Non-trainable params: 84,916,818 (323.93 MB)

None


In [6]:
from tensorflow.keras.optimizers import Adam

lr = 1e-4
model.compile(loss='categorical_crossentropy', optimizer=Adam(lr), metrics=['accuracy'])

batch_size = 32
steps_per_epoch = len(X_train) // batch_size
validation_steps = len(X_test) // batch_size

from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping

best_model_file = "dogs_2.keras"
callbacks = [
    ModelCheckpoint(filepath=best_model_file, verbose=1, save_best_only=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=2, verbose=1, mode='auto', min_lr=1e-6),
    EarlyStopping(monitor='val_loss', patience=7, verbose=1)] # restore_best_weights=True


In [7]:
history = model.fit(
    X_train, y_train, 
    batch_size=batch_size,
    epochs=30, 
    validation_data=(X_test, y_test), 
    steps_per_epoch=steps_per_epoch, 
    validation_steps=validation_steps,  
    callbacks=callbacks,
    verbose=1)

Epoch 1/30
223/223 ━━━━━━━━━━━━━━━━━━━━ 0s 8s/step - accuracy: 0.5937 - loss: 2.7230
Epoch 1: val_loss improved from inf to 2.23053, saving model to dogs_2.keras
223/223 ━━━━━━━━━━━━━━━━━━━━ 2462s 11s/step - accuracy: 0.5942 - loss: 2.7211 - val_accuracy: 0.7595 - val_loss: 2.2305 - learning_rate: 1.0000e-04
Epoch 2/30
  1/223 ━━━━━━━━━━━━━━━━━━━━ 19:57 5s/step - accuracy: 0.6316 - loss: 3.3337

c:\Users\kinga\anaconda3\envs\iowad\Lib\contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)



Epoch 2: val_loss did not improve from 2.23053
223/223 ━━━━━━━━━━━━━━━━━━━━ 12s 31ms/step - accuracy: 0.6316 - loss: 3.3337 - val_accuracy: 0.5926 - val_loss: 4.0040 - learning_rate: 1.0000e-04
Epoch 3/30
223/223 ━━━━━━━━━━━━━━━━━━━━ 0s 7s/step - accuracy: 0.9483 - loss: 0.2803
Epoch 3: val_loss improved from 2.23053 to 2.02827, saving model to dogs_2.keras
223/223 ━━━━━━━━━━━━━━━━━━━━ 2288s 10s/step - accuracy: 0.9483 - loss: 0.2801 - val_accuracy: 0.7937 - val_loss: 2.0283 - learning_rate: 1.0000e-04
Epoch 4/30
  1/223 ━━━━━━━━━━━━━━━━━━━━ 19:14 5s/step - accuracy: 0.9474 - loss: 0.3804
Epoch 4: val_loss improved from 2.02827 to 1.54431, saving model to dogs_2.keras
223/223 ━━━━━━━━━━━━━━━━━━━━ 16s 48ms/step - accuracy: 0.9474 - loss: 0.3804 - val_accuracy: 0.7778 - val_loss: 1.5443 - learning_rate: 1.0000e-04
Epoch 5/30
223/223 ━━━━━━━━━━━━━━━━━━━━ 0s 7s/step - accuracy: 0.9879 - loss: 0.0558
Epoch 5: val_loss did not improve from 1.54431
223/223 ━━━━━━━━━━━━━━━━━━━━ 2291s 10s/step